 # Installation of BeautifulSoup and requests package

In [ ]:
# !pip install requests

In [ ]:
# !pip install BeautifulSoup4

In [ ]:
# !pip install selenium

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import urllib
import time
import pandas as pd
import sys

In [ ]:
from selenium import webdriver
driver = webdriver.Firefox(executable_path=r'P:\Desktop\geckodriver-v0.23.0-win64\geckodriver.exe')
url = "https://inaturalist.nz/observations/18639810"

driver.get(url)

In [ ]:
html = driver.page_source
soup = bs(html,  'lxml')



Wikipedia Scraping

In [ ]:
import wikipedia

In [65]:
# Original GBIF Exotic data in excel format. Reading excel file using pandas 
data = pd.read_excel("T:/projects/2018/SCION/Stream 2/Outer Join Exotic_GBIF Data/181208_Exotic_Gbif_Full_Species.xlsx",
                    sheet_name= 'Text Mining Sheet')

In [66]:
data.columns

Index(['Original_species_name', 'LR_Text', 'BG_Text', 'Wiki_Text', 'Flag',
       'Full Text'],
      dtype='object')

In [67]:
list1 = data['Original_species_name']
list1 = list1.unique()
data['Original_species_name'].nunique()

2641

In [ ]:
# try:
#     wikipedia.page('Acallopais rudis')
# except:
#     print("page not found")

In [ ]:
list1[:10]

# Scraping Wikipedia for decription as it contains information about size and other behavioural properties which might be useful for analysis.

In [ ]:
x = ""
new_x = []
description = []
Distribution_habitat = []
parasite_host = []
pest_species = []
for i in list1:
    try:
        url_link = wikipedia.page(i)
        x = url_link.content
        new_x = x.split("\n")
        try:
            d_index = new_x.index("== Description ==")
            life_index = new_x.index('== Life cycle ==')
            description.append(new_x[d_index:life_index])
            try:
                habitat_index = new_x.index("== Distribution and habitat ==")
                parasite_index = new_x.index("== As parasite host ==")
                Distribution_habitat.append(new_x[habitat_index:parasite_index])
                try:
                    parasite_index = new_x.index("== As parasite host ==")
                    pest_index = new_x.index('== As a pest species ==')
                    parasite_host.append(new_x[parasite_index:pest_index])
                    try:
                        pest_index = new_x.index('== As a pest species ==')
                        pest_species.append(new_x[pest_index:pest_index +1 ])
                    except:
                        pest_species.append("NA")
                        
                        
                except:
                    parasite_host.append("NA")
            except:
                Distribution_habitat.append("NA")
            
            
            
        except:
            description.append("NA")
        
        
    except:
        description.append("page not found")
        

In [ ]:
df_wk = pd.DataFrame({'Original species name':list1})

In [ ]:
df_wk['wiki_description']=description

In [ ]:
df_wk.info()

# Merge Wiki_Description with Exotic_GBIF_All_Data

In [ ]:
data = data.merge(df_wk, on=['Original species name'], how = "left" )
data.info()

# Bug Guide Website Scraping - 

In [ ]:
df = pd.DataFrame({'Original species name':list1})

In [ ]:
df.head(5)

In [ ]:
list1 = data['Original species name']
list1 = list1.unique()

In [ ]:
def get_list_values(text ,value):
    main_list = ''
    temp_list = []

    for list_of_text in text.split("\n"):
        if list_of_text.startswith(value):
            temp_list.append(list_of_text)

    if len(temp_list) != 0:
        main_list = temp_list[0]
    else:
        main_list = "NA"

    return main_list

In [ ]:
def get_contents_from_text(link):
    print(link)
    r = requests.get(link)
    soup = bs(r.text,'lxml')
    text = soup.text
    size.append(get_list_values(text, 'Size'))
    Identification.append(get_list_values(text,'Identification'))
    Range.append(get_list_values(text, 'Range'))
    Season.append(get_list_values(text , 'Season'))
    Life_cycle.append(get_list_values(text , 'Life Cycle'))
    Food.append(get_list_values(text , 'Food'))
    Remarks.append(get_list_values(text , 'Remarks'))


In [ ]:
data_collection = []
size = []
Identification = []
Range = []
Food = []
Remarks = []
Life_cycle = []
Season = []
def create_list_for_link():
    link_list = []
    for link in soup.find_all("a"):
    #print(link.attrs)
        link_list.append(list(link.attrs.values()))
    return link_list

def get_info(index_link):
    #info_link_list = []
    rr = requests.get(index_link)
    soup = bs(rr.text, 'lxml')
    info_link = soup.find('a',string = 'Info')
    if info_link is None:
        info_link = index_link
    else:
        info_link = info_link.get('href')
    return(info_link)


for i in list1:
    x = i.split(" ")
    r= requests.get('https://bugguide.net/index.php?q=search&keys=' + x[0]+ '+' + x[1] + '&search=Search')
    #r= requests.get('https://bugguide.net/index.php?q=search&keys=Acyrthosiphon+kondoi&search=Search')
    soup = bs(r.text, "lxml")
    link_list_new = create_list_for_link()
    #print(link_list_new)

    index_value = link_list_new.index(['https://bugguide.net/adv_search/bgsearch.php'])
    index_link = link_list_new[index_value+1]
    first_link = index_link[0]
    info_link_returned = get_info(first_link)
    get_contents_from_text(info_link_returned)


In [ ]:
https://inaturalist.nz/taxa/search?q=spoladea-recurvalis

In [ ]:
df['BG_size']=size
df['BG_Season']=Season
df['BG_Life_cycle']=Life_cycle
df['BG_Remarks']=Remarks
df['BG_Food'] = Food
df['BG_Identification']=Identification
df['BG_Range']=Range

In [ ]:
df.info()

In [ ]:
data.info()

In [ ]:
data = data.merge(df, on=['Original species name'], how = "left" )
data.head(5)
data.info()

## Saved excel file with added information from bug guide, wikipedia and merged into original file.

In [ ]:
#data.to_excel('T:/projects/2018/SCION/Stream 2/All Exotic GBIF Merged Data/Exotic_Gbif_BG__Wiki_All_Data_Species1.xlsx')

### Calculating size of the bugs which we scraped from bug guide. As it is in text form we, have used regular expression to extract information.

In [ ]:
data = pd.read_excel("T:/projects/2018/SCION/Stream 2/All Exotic GBIF Merged Data/Exotic_Gbif_BG__Wiki_All_Data_Species1.xlsx")

In [ ]:
data.info()

In [ ]:
data['Size(mm)'].unique()

In [ ]:
import numpy as np
data_size_na = data[np.isnan(data['Size(mm)']) == True]
data_size_na['BG_size'].unique()

In [ ]:
size_data = data_size_na[['BG_size','Original species name']].reset_index().drop(['index'], axis =1)
size_data.head()
#pd.isnull(size_data.iloc[]['BG_size'])== True

In [ ]:
pattern = re.compile(r'\d*[.]?\d*[-<]?\d*[.]?\d*')

matches = re.findall(pattern,size_data.iloc[14]['BG_size'])
type(matches)
list1 = []
#print(matches.pop(''))
# matches = pattern.finditer(size_data.iloc[14]['BG_size'])
# print(matches)
for match in matches:
    if match !='':
        list1.append(match)
        print(list1)


In [ ]:
# import re
# pattern = re.compile(r'\d*[.]?\d*[-<]?\d*[.]?\d*')
# list1 = []
# single_list = []
# for i in range(len(size_data['BG_size'])):
#     if pd.isnull(size_data.iloc[i]['BG_size']) == False:
#         matches = re.findall(pattern,size_data.iloc[i]['BG_size'])
#         #print(matches)
#         for match in matches:
#             if match !='':
#                 #print(match)
#                 list1.append(match)
#         single_list.append(list1[0])
                
#     else:
#         continue

In [ ]:
import re

def create_single_list(matches):
    single_list = []
    for match in matches:
        if match != '':
            single_list.append(match)
            #print(single_list[0])
    return single_list[0]
    
pattern = re.compile(r'\d*[.]?\d*[-<]?\d*[.]?\d*')
list1 = []
for i in range(len(size_data['BG_size'])):
    if pd.isnull(size_data.iloc[i]['BG_size']) == False:
        matches = re.findall(pattern, size_data.iloc[i]['BG_size'])
        #print(matches)
        list_value = create_single_list(matches)
        list1.append(list_value)
    else:
        list1.append(None)

    

In [ ]:
list1

In [ ]:
# '<3'.strip('<')
# list1[1].strip('<').split('-')
range(len(list1))
#pd.isnull(list1[14])

In [ ]:
#size_data['BG_avg_Size'] = np.nan
list_new = []

for i in range(len(list1)):
    if pd.isnull(list1[i]) != True:
        x = list1[i].strip('<').split('-')
        if len(x) <2 :
            list_new.append(float(x[0]))
        else:
            avg_size = float(x[0])+float(x[1])/2
            list_new.append(avg_size)
    else:
        list_new.append(np.nan)
        

In [ ]:
size_data['BG_avg_Size'] = list_new

In [ ]:
data_new = pd.merge(data, size_data, how= 'left', on= ['Original species name'])

In [ ]:
data_new.head()

In [ ]:
data_new.to_excel('T:/projects/2018/SCION/Stream 2/All Exotic GBIF Merged Data/Exotic_Gbif_BG__Wiki_All_Data_Species1.xlsx')

In [ ]:
# Wikipedia Scraping on entire list

In [9]:
list1

array(['Abgrallaspis cyanophylli', 'Acallopais rudis',
       'Acantholybas brunneus', ..., 'Pediculus capitis',
       'Pseudoscorpionida', 'Liposcelis divinatorius'], dtype=object)

In [69]:
text_wiki = []

home_dir = "T:/projects/2018/SCION/Stream 2/text mining"
for name in list1:
    text_from_wiki = ''
    if len(name.split())>1:
        species_name = name.split()[0] + '_' + name.split()[1]
    else:
        species_name = name
    #print(species_name)
    r = requests.get("https://en.wikipedia.org/wiki/"+species_name)
    soup = bs(r.text, 'lxml')
    for text in soup.find_all('p'):
        text_from_wiki += text.getText()
        if len(text.getText()) != 45:
            with open('{}/{}.txt'.format(home_dir, species_name),'a+', encoding= 'utf-8') as f:
                a = text.getText()
                #print(a)
                f.write(a)
        elif len(text.getText()) == 45:
            with open('{}/0_Text_{}.txt'.format(home_dir, species_name),'a+', encoding= 'utf-8') as f:
                a = text.getText()
                #print(a)
                f.write(a)
    text_wiki.append(text_from_wiki)    

FileNotFoundError: [Errno 2] No such file or directory: 'T:/projects/2018/SCION/Stream 2/text mining/0_Text_Priophorus_brullei/morio.txt'

In [ ]:
import io

with io.open('log.txt', 'a', encoding='utf8') as logfile:
    for tr in soup.find_all('tr')[2:]:
        tds = tr.find_all('td')
        logfile.write(u"%s, %s, %s\n" % (tds[0].text, tds[1].text, tds[2].text))

In [58]:
len(text_wiki)

10